In [8]:
import tempfile
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

'''
簡單CNN模型
'''

mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28,28)),
    keras.layers.Reshape(target_shape=(28,28,1)),
    keras.layers.Conv2D(filters=12, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(10)
])
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=4, validation_split=0.1)
_, baseline_model_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('accuracy:', baseline_model_accuracy)
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('save as', keras_file)

Epoch 1/4
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2911 - accuracy: 0.9174 - val_loss: 0.1154 - val_accuracy: 0.9698
Epoch 2/4
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1102 - accuracy: 0.9679 - val_loss: 0.0832 - val_accuracy: 0.9788
Epoch 3/4
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0804 - accuracy: 0.9762 - val_loss: 0.0703 - val_accuracy: 0.9807
Epoch 4/4
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0668 - accuracy: 0.9804 - val_loss: 0.0654 - val_accuracy: 0.9825
accuracy: 0.9775000214576721
save as C:\Users\minku\AppData\Local\Temp\tmp9r6ig64x.h5


C:\Users\minku\AppData\Local\Temp\ipykernel_9408\2636089171.py:30: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [9]:
'''
使用剪枝技術微調訓練好的模型
'''
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
batch_size = 128
epochs = 2
validation_split = 0.1
num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50, final_sparsity=0.8, begin_step=0, end_step=end_step)
}
model_for_pruning = prune_low_magnitude(model, **pruning_params)
model_for_pruning.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
]
model_for_pruning.fit(train_images, train_labels, batch_size=batch_size, epochs=epochs, validation_split=validation_split, callbacks=callbacks)
_, model_for_pruning_accuracy = model_for_pruning.evaluate(test_images, test_labels, verbose=0)
print('baseline accuracy:', baseline_model_accuracy) 
print('pruning accuracy:', model_for_pruning_accuracy)

Epoch 1/2
422/422 [==============================] - 19s 18ms/step - loss: 0.0861 - accuracy: 0.9760 - val_loss: 0.0977 - val_accuracy: 0.9750
Epoch 2/2
422/422 [==============================] - 7s 16ms/step - loss: 0.1058 - accuracy: 0.9701 - val_loss: 0.0893 - val_accuracy: 0.9763
baseline accuracy: 0.9775000214576721
pruning accuracy: 0.972599983215332


In [12]:
'''
使用剪枝技術將模型缩小3倍
'''

import os
import zipfile

def get_gzipped_model_size(file):
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression = zipfile.ZIP_DEFLATED) as f:
        f.write(file)
    return os.path.getsize(zipped_file)

model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Save pruned Keras model to:', pruned_keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()
_, pruned_tflite_file = tempfile.mkstemp('.tflite')
with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)
print('Save pruned TFLite model to:', pruned_tflite_file)

print("gzip baseline Keras model size: %.2f bytes" % (get_gzipped_model_size(keras_file))) #原始model
print("gzip pruned Keras model size: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file))) #model after purning
print("gzip pruned TFlite model size: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file))) #TFlite type


C:\Users\minku\AppData\Local\Temp\ipykernel_9408\4091028920.py:16: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Save pruned Keras model to: C:\Users\minku\AppData\Local\Temp\tmp91f0s6u7.h5
INFO:tensorflow:Assets written to: C:\Users\minku\AppData\Local\Temp\tmpb1d1hqpi\assets


INFO:tensorflow:Assets written to: C:\Users\minku\AppData\Local\Temp\tmpb1d1hqpi\assets


Save pruned TFLite model to: C:\Users\minku\AppData\Local\Temp\tmpx54doxm2.tflite
gzip baseline Keras model size: 78331.00 bytes
gzip pruned Keras model size: 25867.00 bytes
gzip pruned TFlite model size: 25073.00 bytes


In [13]:
'''
使用剪枝和量化將模型缩小 10 倍
'''
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()
_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')
with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)
print('quantized and pruned TFLite model save to:', quantized_and_pruned_tflite_file)
print("gzip baseline Keras model size: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("gzipped quantized and pruned TFlite model size: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

INFO:tensorflow:Assets written to: C:\Users\minku\AppData\Local\Temp\tmp4mm4w9jk\assets


INFO:tensorflow:Assets written to: C:\Users\minku\AppData\Local\Temp\tmp4mm4w9jk\assets


quantized and pruned TFLite model save to: C:\Users\minku\AppData\Local\Temp\tmpnag810_9.tflite
gzip baseline Keras model size: 78331.00 bytes
gzipped quantized and pruned TFlite model size: 8254.00 bytes
